## SVM

### Code

In [ ]:
import numpy as np
import pandas as pd
import math
import random
from sklearn.model_selection import train_test_split

In [ ]:
class SVM:
    def __init__(self, X, y, sigma = 10, C = 200, toler = 0.001):
        self.X = X
        self.y = np.ravel(y)
        self.numSamples = X.shape[0]
        self.sigma = sigma  # 高斯核函数的sigma参数
        self.C = C    # 惩罚参数
        self.toler = toler    # 松弛变量
        self.b = 0    # 偏置b
        self.alphas = np.zeros(self.X.shape[0])    # 储存α的数组
        self.E = np.zeros(self.X.shape[0])   # 储存E的数组 
        self.kernelMatrix = self.calc_Kernel()    # 储存核函数值的矩阵

    # 根据公式7.66和7.90计算高斯核
    def calc_Kernel(self):
        k_matrix = np.mat(np.zeros((self.numSamples, self.numSamples)))
        for i in range(self.numSamples):
            x = self.X[i,:]
            for j in range(i, self.numSamples):
                z = self.X[j,:]
                r = (x - z) * (x - z).T
                gsi = np.exp(-1 * r / (2 * self.sigma ** 2))
                k_matrix[i,j] = gsi
                k_matrix[j,i] = gsi
        return k_matrix

    # 根据公式7.104和7.105计算
    def calc_E(self, i):
        gx = np.multiply(self.alphas, self.y) * self.kernelMatrix[:,i] + self.b
        Ei = gx - self.y[i]
        return Ei
        
    # 选择变量2，采用|E2-E1|最大原则
    def select_AlphaJ(self, i, E1):
        E2 = 0
        maxE1_E2 = -1
        maxIndex = -1
        
        # 只看E>0的
        nozeroE = [i for i, Ei in enumerate(self.E) if Ei != 0]

        if (len(nozeroE) > 0):
            for j in nozeroE:
                if j == i: continue
                Ej = self.calc_E(j)
                if math.fabs(E1 - Ej) > maxE1_E2:
                    maxE1_E2 = math.fabs(E1 - Ej)
                    E2 = Ej
                    maxIndex = j
        else:
            maxIndex = i
            while maxIndex == i:
                maxIndex = int(random.uniform(0, self.numSamples))
            E2 = self.calc_E(maxIndex)
        return maxIndex, E2

    # 先外循环找变量1再内循环找变量2
    def Loop(self, i):
        E1 = self.calc_E(i)
        
        # 寻找不满足KKT条件的点，这里我也是学了他人的取巧方式，具体推理可看上面那篇文章
        if (self.y[i] * E1 < -self.toler and self.alphas[i] < self.C) or (self.y[i] * E1 > self.toler and self.alphas[i] > 0):
            j,E2 = self.select_AlphaJ(i, E1)
            alpha_old_1 = self.alphas[i]
            alpha_old_2 = self.alphas[j]
            y1 = self.y[i]
            y2 = self.y[j]
            
            # 有图7.8所得
            if y1 != y2:
                L = max(0, alpha_old_2 - alpha_old_1)
                H = min(self.C, self.C + alpha_old_2 - alpha_old_1)
            else:
                L = max(0, alpha_old_2 + alpha_old_1 - self.C)
                H = min(self.C, alpha_old_2 + alpha_old_1)
            
            # 说明无法再优化
            if L == H:
                return 0
            # 公式7.106 7.107
            alpha_new_2 = alpha_old_2 + y2 * (E1 - E2) / (self.kernelMatrix[i,i] + self.kernelMatrix[j,j] - 2 * self.kernelMatrix[i,j])
                
            # 公式7.108
            if (alpha_new_2 > H):
                alpha_new_2 = H
            elif (alpha_new_2 < L):
                alpha_new_2 = L
                        
            # 公式7.109
            alpha_new_1 = alpha_old_1 + y1 * y2 * (alpha_old_2 - alpha_new_2)
            
            # 公式7.115 7.116
            b1_new = -1 * E1 - y1 * self.kernelMatrix[i,i] * (alpha_new_1 - alpha_old_1) - y2 * self.kernelMatrix[j,i] * (alpha_new_2 - alpha_old_2) + self.b
            b2_new = -1 * E2 - y1 * self.kernelMatrix[i,j] * (alpha_new_1 - alpha_old_1) - y2 * self.kernelMatrix[j,j] * (alpha_new_2 - alpha_old_2) + self.b
                        
            if (alpha_new_1 > 0) and (alpha_new_1 < self.C):
                b_new = b1_new
            elif (alpha_new_2 > 0) and (alpha_new_2 < self.C):
                b_new = b2_new
            else:
                b_new = (b1_new + b2_new) / 2
                    
            # 更新a1 a2以及E1 E2
            self.alphas[i] = alpha_new_1
            self.alphas[j] = alpha_new_2
            self.b = b_new
            self.E[i] = self.calc_E(i)
            self.E[j] = self.calc_E(j)
                
            if (math.fabs(alpha_new_2 - alpha_old_2) < 0.00001):
                return 0
            else:
                return 1
            
        else:
            return 0

    def train(self, max_iter = 1000):
        alpha_change = 1
        iter = 0
        entireSet = True
        
        while (iter < max_iter) and (alpha_change > 0) or entireSet:
            alpha_change = 0
            iter += 1
            # 先过一遍全部样本，然后就先支持向量的样本，都满足的KKT的话，再全样本
            if entireSet:
                for i in range(self.numSamples):
                    alpha_change += self.Loop(i)
                    print("iter:", iter, "i value:", i, "entire_Set, alpha changed:", alpha_change, "b:", self.b)
            else:
                support_vector_i = [i for i, alpha in enumerate(self.alphas) if alpha > 0 * alpha < self.C]
                for i in support_vector_i:
                    alpha_change += self.Loop(i)
                    print("iter:", iter, "i value:", i, "support_vector_Set, alpha changed:", alpha_change, "b:", self.b)
                    
            if entireSet:
                entireSet = False
            elif alpha_change == 0:
                entireSet = True
                
    #单个核函数计算函数，用于公式7.90
    def calcKernelValue(self, xj, xi):
        r = (xj - xi) * (xj - xi).T
        gsi = np.exp(-1 * r / (2 * self.sigma ** 2))
        return gsi
    
    # 预测函数
    def predict(self, test_X):
        support_vector = np.nonzero(self.alphas > 0)[0]
        gx = 0
        for i in support_vector:
            kernel_value = self.calcKernelValue(self.X[i,:], test_X)
            gx += self.alphas[i] * self.y[i] * kernel_value
        gx += self.b
        return np.sign(gx)

### 训练集

In [ ]:
dataset = pd.read_table("testSet.txt", header = None)
dataset = np.mat(dataset)
X = dataset[:,:-1]
y = dataset[:,-1]
y[y == 0] = -1

train_X, test_X, train_y, test_y = train_test_split(X, y, test_size = 0.2, random_state = 123456)

svm = SVM(X=train_X, y=train_y, C=10)
svm.train()

error = 0
for i in range(test_X.shape[0]):
    p = svm.predict(test_X[i])
    if p != test_y[i]:
        error += 1

print(error / len(test_y) * 100)